In [1]:
# This notebook aggregates all output from toboggan for an input dataset
# and then creates a runtime histogram for that dataset

# CHANGE froot TO THE DATASET YOU WANT: zebra, human, mouse
froot = 'zebra'
timeoutexceed = 800
which_alg = "toboggan"

In [2]:
inputfile = "all-" + froot + ".txt"

import table_generator

datadict, datamatrix = table_generator.make_tables(inputfile)

In [3]:
num_trivial = 0
num_timedout = 0
time_totals = []
total_num = len(datamatrix)
for key, val in datadict.items():
    row = datamatrix[val]
    """
    datamatrix[j] = [ n, m, n_red, m_red,
                      k_groundtruth, k_opt, cutset_bound, improved_bound,
                      t_w, t_path]
    """
    if row[2] == '1':  # trivial instance because reduced graph has 1 node
        num_trivial += 1
        continue
    elif row[5] == 'None':  # timedout instance
        num_timedout += 1
        continue
    else:
        time_total = float(row[8]) + float(row[9])
        if float(row[8]) > timeoutexceed:
            #print("time value exceeds timeout")
            print(key)
            #print(row)
            continue
        if time_total == 0.0:
            print("Time of 0.0, something went wrong")
            print(froot + " " + key)
            print(row)
        time_totals.append(time_total)

In [4]:
table_generator.print_data_summary( froot, len(datadict), num_trivial )
table_generator.print_alg_summary( which_alg, timeoutexceed, num_timedout, len(time_totals) )

zebra has 
	total   instances:      1549373
	trivial instances:      1103493
	nontriv instances:       445880

toboggan with 800s timeout has
	timeout instances:          178
	success instances:       445702


In [7]:
import matplotlib.pyplot as plt
import numpy as np

plt.clf()
#logbins=np.logspace(-4, 3, 7)
logbins = [10**-4, 10**-3, 10**-2, 10**-1, 1, 10, 100, 1000, 10000]
n, bins, patchs = plt.hist(time_totals, bins=logbins, log=True)
plt.title("Runtimes: {}, {}".format(which_alg, froot))
plt.xlabel("time")
plt.ylabel("Frequency")
plt.gca().set_xscale("log")

#plt.show()
plt.savefig( ''.join(['runtime-toboggan-', froot, '.eps']), format='eps', dpi=300)
plt.savefig( ''.join(['runtime-toboggan-', froot, '.png']), format='png', dpi=300)
plt.close()

In [6]:
max(time_totals)

1546.4690294266